In [ ]:
from pathlib import Path
import numpy as np
from aicsimageio import AICSImage
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

%matplotlib notebook
%matplotlib inline
import pandas as pd
import seaborn as sns

import sys
src_path = str(Path.cwd().parent)
if src_path not in sys.path:
    sys.path.append(src_path)
#from src.d00_utils.dirnames import excl_analysis_dirname
from src.d00_utils import utilities as utils
from src.d00_utils import dirnames as dn
from src.d01_init_proc import vis_and_rescale

In [ ]:
input_dirpath = input('Please enter the full path for the input directory:\n')
input_dirpath = Path(input_dirpath)

In [ ]:
caax_ch = 0
cell_ch = 2

ch_labels = ['mRuby', 'GFP', 'membrane', 'phalloidin']
ch_save_abbr = ch_labels

In [ ]:
def save_ratiometric_img(num_ch, denom_ch, img, saveinfo, ratio_thresh=1.1):
    
    (imgname, ch_labels, ch_save_abbr, output_subdirpath) = saveinfo
    info_df = utils.extract_img_info(imgname)
    
    # convert 0s in the image to nan
    img_nanbg = np.where(img==0, np.nan, img)
    
    # calculate ratio
    if num_ch is None:
        ratio = 1 / img_nanbg[:, denom_ch, :, :, :].squeeze()
    else:
        ratio = img_nanbg[:, num_ch, :, :, :].squeeze() / img_nanbg[:, denom_ch, :, :, :].squeeze()
    
    # set mean = 1
    ratio = ratio / np.nanmean(ratio)
    bin_ratio = (ratio > ratio_thresh).astype('bool')
    
    # Set labels
    base_imgname = imgname.split('.')[0]
    UID = info_df['UID'].values[0]
    if num_ch is None:
        num_lab = 1
    else:
        num_lab = {ch_labels[num_ch]}
    
    
    # show image
    fig, axs = plt.subplots(1, 2)
    # #rg = axs[0].imshow(ratio, norm=mcolors.TwoSlopeNorm(vmin=0,
    #                               vcenter=1, vmax=3), cmap='coolwarm', interpolation=None)
    denom_img, _ = vis_and_rescale.rescale_img(img[:, denom_ch, np.newaxis, :, :, :], conv_to_8bit=True)
    axs[0].imshow(denom_img.squeeze(), cmap='gray', interpolation=None)
    axs[0].set_title(f'Rescaled ch{denom_ch} image')
    axs[0].axis('off')
    #plt.colorbar(rg, ax=axs[0], shrink=0.5)
    axs[1].imshow(bin_ratio, cmap='gray', interpolation=None)
    axs[1].axis('off')
    axs[1].set_title(f'normalized {num_lab} / {ch_labels[denom_ch]} > {ratio_thresh}')
    UID = info_df['UID'].values[0]
    plt.suptitle(UID)
    plt.tight_layout()
    plt.show()
    
    # Save figures
    output_figspath = output_subdirpath / 'figs'
    output_figspath.mkdir(parents=True, exist_ok=True)
    fig.savefig(output_figspath / f'{base_imgname}.pdf')
    
    # Save ratiometric array
    output_arraypath = output_subdirpath / 'ratiometric_arrays'
    output_arraypath.mkdir(parents=True, exist_ok=True)
    np.save(output_arraypath / f'{base_imgname}.npy', ratio)
    return ratio, bin_ratio

In [ ]:
input_dirpath = Path(input_dirpath)

# Get output directory path
proc_dirpath = utils.get_proc_dirpath(input_dirpath)
output_dirpath = proc_dirpath / dn.analyses_dirname / dn.ratiometric_dirname

imgpaths = [path for path in input_dirpath.glob('*.ome.tif')]
imgpaths.sort()


imgnames = []
cellareas = []
excludedareas = []

for imgpath in imgpaths:
    imgnames.append(imgpath.name)
        
    # Get output subdirectory paths
    output_subdirpath = output_dirpath / f'exclusion_caaxch{caax_ch}'

    saveinfo = (imgpath.name, ch_labels, ch_save_abbr, output_subdirpath)

    img_file = AICSImage(imgpath, reader=OmeTiffReader)
    img = img_file.data
    img = img[0:1, :, :, :, :]

    cellarea = np.count_nonzero(img[:, cell_ch, :, :, :])
    cellareas.append(cellarea)

    _, bin_ratio = save_ratiometric_img(None, caax_ch, img, saveinfo)
    excludedarea = np.count_nonzero(bin_ratio)
    excludedareas.append(excludedarea)
        
df = pd.DataFrame({'Image name': imgnames, 'Cell area': cellareas, 'Excluded area': excludedareas})
df['% exclusion'] = df['Excluded area'] / df['Cell area']
df.to_csv(output_dirpath / (f'ratiometric_exclusion_caaxch{caax_ch}.csv'), index=False)
print("Done")

In [ ]:
def add_info(df):
    
    # Split imgname by '_'
    namesplits = [imgname.split('_') for imgname in df['Image name']]
    
    # Obtain dish name/number
    df['dish'] = [namesplit[1] for namesplit in namesplits]

    # Obtain well info
    df['well'] = [imgname.split('_')[5].split('-')[1] for imgname in df['Image name']]
    
    df['unique well'] = df['dish'] + '-' + df['well']
    
    return df


In [ ]:
x_label = 'well'
y_labels = ['Cell area', '% exclusion']
sns.swarmplot(df, x=x_label, y='% exclusion', legend=False, color='gray')

In [ ]:
df0_path = Path('/Users/kwu2/Library/CloudStorage/GoogleDrive-kwu2@stanford.edu/My Drive/Lab/CryoEM_live_imaging/Experiments/CE027/actin genetic exp/CE027_ratB_div7/aligned/experiment/I7_ratB_div7_and_I7_ratB_div7_phalloidin_aligned/img_processing/analyses/ratiometric/ratiometric_exclusion_caaxch0.csv')
df1_path = Path('/Users/kwu2/Library/CloudStorage/GoogleDrive-kwu2@stanford.edu/My Drive/Lab/CryoEM_live_imaging/Experiments/CE027/actin genetic exp/CE027_ratB_div7/aligned/experiment/I7_ratB_div7_and_I7_ratB_div7_phalloidin_aligned/img_processing/analyses/ratiometric/ratiometric_exclusion_caaxch1.csv')

In [ ]:
df2_path = Path('/Users/kwu2/Library/CloudStorage/GoogleDrive-kwu2@stanford.edu/My Drive/Lab/CryoEM_live_imaging/Experiments/CE027/actin genetic exp/CE027_ratB_div7/aligned/experiment/I8_ratB_div7_and_I8_ratB_div7_phalloidin_A1A2_aligned/img_processing/analyses/ratiometric/ratiometric_exclusion_caaxch0.csv')

In [ ]:
df0 = pd.read_csv(df0_path)
df1 = pd.read_csv(df1_path)
df2 = pd.read_csv(df2_path)

df_combined = pd.concat([df0, df1, df2], ignore_index=True)
df_combined.head()


In [ ]:
df_combined = add_info(df_combined)
df_combined.head()

In [ ]:
x_label = 'unique well'
y_labels = ['Cell area', '% exclusion']
for y_label in y_labels:
    sns.swarmplot(df_combined, x=x_label, y=y_label, legend=False, color='gray')
    plt.show()


In [ ]:
df = df_combined

In [ ]:
unique_wells = ['I7-A1', 'I7-A2', 'I7-A3', 'I7-A4', 'I7-B1', 'I7-B2', 'I7-B3', 'I7-B4', 'I8-A1', 'I8-A2']
unique_well_df = [df['unique well'] == well for well in unique_wells]
conditions = ['281', '218+424', '281', '218+424', '281+351', '281+351', '218+281', '218+281', '283+218', '283+218']
replicates = [1, 1, 2, 2, 1, 2, 1, 2, 1, 2]

df_combined['condition'] = np.select(unique_wells_df, conditions)
df_combined['replicate'] = np.select(unique_wells_df, replicates)
df.head()




In [ ]:
x_label = 'condition'
y_labels = ['Cell area', '% exclusion']
hue = 'replicate'
for y_label in y_labels:
    sns.swarmplot(df, x=x_label, y=y_label, hue='replicate', legend=False, color='gray')
    plt.show()


In [ ]:
df.groupby('unique well')['unique well'].count()